In [104]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [105]:
from texas_api import TexasApi

In [106]:
tx = TexasApi()

In [93]:
venues_lt = tx.find_by_lt(1, 2018, 20000)

In [98]:
# venues_lt

In [99]:
scoped_venues = tx.scoped(venues_lt)

In [100]:
scoped_venues[:3]

[{'taxpayer_number': '32048438157', 'total_receipts': '13611'},
 {'taxpayer_number': '17426162909', 'total_receipts': '2354'},
 {'taxpayer_number': '32044790700', 'total_receipts': '8531'}]

In [22]:
numbers_lt = [venue['taxpayer_number'] for venue in scoped_venues]

In [23]:
len(numbers_lt)

536

In [124]:
venues = tx.find_by_gt(1, 2020, 30000, 'DALLAS')

In [126]:
# venues

In [127]:
scoped_venues = tx.scoped(venues)

In [128]:
len(scoped_venues)

609

In [26]:
numbers_gt = [venue['taxpayer_number'] for venue in scoped_venues]

In [28]:
inter = list(set(numbers_gt) & set(numbers_lt))

In [29]:
len(inter)

32

In [208]:
def venue_ids(venues):
    return [{'taxpayer_number': venue['taxpayer_number'], 'location_number': venue['location_number']} for venue in venues]

In [237]:
def find_intersection(lt_venues, gt_venues):
    intersect = []
    for venue in lt_venues:
        if venue in gt_venues:
            intersect.append(venue)
    return intersect

In [248]:
import pdb
def find_gap(month, begin_year, end_year, less_than, greater_than, location = None):
    lt_venues = tx.find_by_lt(month, begin_year, less_than, location)
    gt_venues = tx.find_by_gt(month, end_year, greater_than, location)
    combined_venues = find_intersection(venue_ids(lt_venues), venue_ids(gt_venues))
    return combined_venues

In [239]:
def ranges(start, end, gap_size):
    return [(num, num + gap_size) for num in range(start, end, gap_size)]

In [240]:
import itertools

def find_leapers(gap_amount, end, month, begin_year, end_year, city):
    leapers = []
    for lt, gt in ranges(gap_amount, end, gap_amount):
        leapers.append(find_gap(month, begin_year, end_year, lt, gt, city))
    total_leapers = list(itertools.chain(*leapers))
    return total_leapers

In [230]:
cities = ["houston", "san antonio", "dallas", "austin", "fort worth", "el paso", "arlington"]
upper_cities = [city.upper() for city in cities]

In [253]:
total_leaps = [find_leapers(10000, 60000, 1, 2018, 2020, city) for city in upper_cities]

In [249]:
len(total_leaps)

7

In [254]:
total_vens = list(itertools.chain(*total_leaps))

In [255]:
dict_vens = [dict(ven) for ven in total_vens]

In [256]:
len(dict_vens)

342

In [217]:
import pandas as pd

In [257]:
df = pd.DataFrame(dict_vens)

In [258]:
df.to_csv('./gap_venues.csv')

In [32]:
top_vens = tx.find_by_gt('DALLAS', 1, 2018, 20000)

In [33]:
len(top_vens)

715